In [1]:
from tqdm import tqdm
import pandas as pd

In [2]:
df_articles = pd.read_parquet('../newspaper_part1.parquet')

In [7]:
df_articles = df_articles[df_articles.body != '']

In [3]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification", 
                      model="BaptisteDoyen/camembert-base-xnli", device="cuda:0", framework='pt')

sequence = df_articles.body.iloc[30]#"Qui sont les sources d’inspirations des grands décideurs économiques de ce monde? C’est la question posée par le cabinet PricewaterhouseCoopers à 1.330 dirigeants dans 68 pays dans le cadre de la 16e édition de son étude mondiale «CEO Survey».\xa0Et le résultat est pour le moins surprenant: au total, 60 % des dirigeants ont choisi un homme politique d’après-guerre ou un commandant militaire. Respectivement en 2e et 5e positions, Steve Jobs (ex-PDG d’Apple) et Jack Welch (ex-General Electric) sont les deux seuls hommes d’affaires et l’ex-Première ministre britannique Margaret Thatcher est la seule femme.\xa01. Winston Churchill2. Steve Jobs3. Mahatma Gandhi4. Nelson Mandela5. Jack Welch6. Abraham Lincoln7. Margaret Thatcher8. Ronald Reagan9. John F. Kennedy10. Napoléon Bonaparte/Bill Clinton«Certains leaders sont universels. Dans une situation tendue, les dirigeants choisissent pour modèle des leaders qui ont su faire preuve de persévérance face à l’adversité et conduire des changements importants», explique dans un communiqué Laurent Acharian, senior manager chez PwC et auteur de Cent jours des dirigeants, l’état de grâce existe-t-il?"
candidate_labels = ['environnement', 'sport', 'economie', 'divertissement', 'high-tech', 'politique', "justice", "monde", "société", "santé", "faits divers"]
hypothesis_template = "Ce texte parle de {}."    

classifier(sequence, candidate_labels, hypothesis_template=hypothesis_template)

{'sequence': 'Le président Donald Trump a suggéré de lâcher des bombes nucléaires sur les ouragans avant qu’ils ne touchent les Etats-Unis, a affirmé dimanche le site d’information américain Axios.Selon Axios, Donald Trump a demandé lors d’une réunion sur les ouragans s’il serait possible, pour les empêcher de se former complètement en mer, de lâcher une bombe atomique sur leur centre. Selon une source anonyme citée par Axios, les personnes qui participaient à la réunion en sont sorties perplexes. Le site ne précise pas quand cette réunion aurait eu lieu.Consensus scientifique pour dire que cela ne marcherait pasL’idée avancée par Donald Trump n’est pas nouvelle, précise toutefois Axios. Elle a été initialement émise dans les années 1950 par un scientifique qui travaillait pour le gouvernement américain, sous la présidence de Dwight Eisenhower. Depuis, l’idée ressurgit régulièrement, bien qu’il y ait un consensus scientifique pour dire que cela ne marcherait pas.Selon Axios, Donald Tru

In [8]:
test_dataset = df_articles.sample(200)

In [14]:
# Create test and train dataframes
from sklearn.model_selection import train_test_split
dict_labels = {'planete': 0, 'sport': 1, 'economie': 2, 'sciences': 3, 'high-tech': 4, 'politique': 5}
p_train, p_test = train_test_split(df_articles[(df_articles.category_id == 'planete')], test_size=1000, train_size=3000, random_state=42)
s_train, s_test = train_test_split(df_articles[(df_articles.category_id == 'sport')], test_size=1000, train_size=3000, random_state=42)
e_train, e_test = train_test_split(df_articles[(df_articles.category_id == 'economie')], test_size=1000, train_size=3000, random_state=42)
sc_train, sc_test = train_test_split(df_articles[(df_articles.category_id == 'sciences')], test_size=500, train_size=2000, random_state=42)
h_train, h_test = train_test_split(df_articles[(df_articles.category_id == 'high-tech')], test_size=1000, train_size=3000, random_state=42)
po_train, po_test = train_test_split(df_articles[(df_articles.category_id == 'politique')], test_size=1000, train_size=3000, random_state=42)
train_dataset = pd.concat([p_train, s_train, e_train, sc_train, h_train, po_train])[['title', 'category_id', 'body']]
train_dataset['label'] = train_dataset.apply(lambda x: dict_labels[x['category_id']], axis=1)
test_dataset = pd.concat([p_test, s_test, e_test, sc_test, h_test, po_test])[['title', 'category_id', 'body']]
test_dataset['label'] = test_dataset.apply(lambda x: dict_labels[x['category_id']], axis=1)

In [9]:
text = test_dataset['body'].to_list()
titles = test_dataset['title'].to_list()
# labels = test_dataset['label'].to_list()
# dict_labels = {'climat': 0, 'sport': 1, 'economie': 2, 'sciences': 3, 'high-tech': 4, 'politique': 5}
# inv_dic = {v:k for (k, v) in dict_labels.items()}

In [68]:
from torch.utils.data import Dataset
class ListDataset(Dataset):
    def __init__(self, original_list):
        self.original_list = original_list
        super(Dataset, self).__init__()

    def __len__(self):
        return len(self.original_list)

    def __getitem__(self, i):
        return self.original_list[i]

dataset = ListDataset(text[:100])

In [86]:
for i in range(len(text)):
    if text[i] == '':
        text[i] = "divers"

In [10]:
body_pred = classifier(text, candidate_labels, hypothesis_template=hypothesis_template, batch_size=32)

In [27]:
reshape_pred = []
for pred in tqdm(body_pred):
    dictio = {}
    dictio["sequence"] = pred["sequence"]
    for i in range(len(pred["labels"])):
        dictio[pred["labels"][i]] = pred["scores"][i]

    reshape_pred.append(dictio)


df_pred = pd.DataFrame.from_dict(reshape_pred)
df_pred

100%|██████████| 200/200 [00:00<00:00, 199966.82it/s]


,sequence,société,politique,faits divers,economie,monde,justice,divertissement,high-tech,environnement,santé,sport
0,Le gouvernement va prolonger jusqu'à fin 2016 ...,0.160894,0.151129,0.120805,0.112044,0.110255,0.106178,0.055357,0.053085,0.048505,0.043062,0.038685
1,Et si on revoyait Adrien Rabiot un jour en équ...,0.079383,0.166596,0.045381,0.010150,0.225728,0.076549,0.069695,0.013730,0.010265,0.040240,0.262284
2,"Grzegorz Rosinski, dessinateur de « Thorgal »....",0.130738,0.037834,0.105463,0.025848,0.165833,0.119166,0.199553,0.023664,0.038155,0.121533,0.032212
3,Muet depuis que la justice sportive l'a dépoui...,0.078327,0.074709,0.149955,0.034273,0.099590,0.178121,0.052264,0.035178,0.042260,0.038369,0.216953
4,De notre envoyé spécial à la cour d’assises de...,0.121580,0.061558,0.261256,0.028771,0.130029,0.200895,0.070252,0.022904,0.049719,0.029477,0.023558
...,...,...,...,...,...,...,...,...,...,...,...,...
195,"Top Chef chez Philippe de Villiers. Lundi, le ...",0.114075,0.091509,0.100108,0.029914,0.128122,0.074495,0.300896,0.038633,0.024697,0.035488,0.062063
196,"Pour ses séries, Arte vise la singularité « av...",0.102904,0.064529,0.069280,0.064909,0.098350,0.115752,0.179566,0.062732,0.084548,0.099262,0.058168
197,"« Quand on est dans la rue, on doit toujours r...",0.126933,0.277954,0.076400,0.045885,0.141132,0.089206,0.065391,0.056187,0.051494,0.035667,0.033751
198,"Le président de Danone, Franck Riboud, a décla...",0.329342,0.281647,0.057144,0.024642,0.107839,0.033940,0.026570,0.055152,0.009565,0.061760,0.012399


In [15]:
import numpy as np
def extract_distrib(proba):
    hist_data=list()
    group_labels = list()
    sorted_proba = np.sort(proba, axis=1)
    for i in range(11):
        hist_data.append(sorted_proba[:, i])
        group_labels.append("Group " + str(i+1))
    return hist_data, group_labels, sorted_proba

In [18]:
prob = df[df.columns[1:]].values

In [23]:
import plotly.express as px

hist_data, group_labels, sorted_proba = extract_distrib(prob)

df = pd.DataFrame(data=sorted_proba, columns=group_labels)
fig = px.histogram(df, x="Group 10", histfunc="count",
                   marginal="box", log_x=True)
fig.show()

In [30]:
dictio = {0: "planete", 1: "sport", 2: "economie",  3: "arts-stars", 4: "high-tech", 5: "politique", 6: "monde", 7: "societe", 8: "faits_divers", 9: "sante", 10: "justice"}

def extract_class(probabilities, th=0.05):
    cat = list()
    if probabilities[0] >= th:
        cat.append("planete")
    if probabilities[1] >= th:
        cat.append("sport")
    if probabilities[2] >= th:
        cat.append("economie")
    if probabilities[3] >= th:
        cat.append("arts-stars")
    if probabilities[4] >= th:
        cat.append("high-tech")
    if probabilities[5] >= th:
        cat.append("politique")
    if probabilities[6] >= th:
        cat.append("monde")
    if probabilities[7] >= th:
        cat.append("societe")
    if probabilities[8] >= th:
        cat.append("faits_divers")
    if probabilities[9] >= th:
        cat.append("sante")
    if probabilities[10] >= th:
        cat.append("justice")
    if cat == []:
        i = np.argmax(probabilities)
        cat.append(dictio[i])
    return " ".join(cat)

In [31]:
df_pred["predicted_class"] = df_pred[["environnement", "sport", "economie", "divertissement", "high-tech", "politique", 'monde', 'société', 'faits divers', 'santé', 'justice']].apply(lambda x: extract_class(x.values), axis=1)

In [33]:
df_pred.to_parquet("zero_shot.parquet")

In [37]:
title_pred = classifier(titles, candidate_labels, hypothesis_template=hypothesis_template, batch_size=32)

In [88]:
import numpy as np
np.save("body_pred.npy", body_pred, allow_pickle=True)
np.save("title_pred.npy", title_pred, allow_pickle=True)

In [162]:
import numpy as np
import more_itertools

predictions = list()
for i in range(len(body_pred)):
  labels_title = title_pred[i]["labels"]
  scores_title = title_pred[i]["scores"]
  labels_body = body_pred[i]["labels"]
  scores_body = body_pred[i]["scores"]
  (labels_title, scores_title) = more_itertools.sort_together([labels_title, scores_title])
  (labels_body, scores_body) = more_itertools.sort_together([labels_body, scores_body])
  r = 1
  scores = r*np.array(scores_body) + (1-r)*np.array(scores_title)
  prediction = list()
  for i in range(len(scores)):
    if scores[i] > 0.2:
      prediction.append(labels_title[i])
  if prediction == []:
    prediction.append(labels_title[np.argmax(scores)])
  predictions.append(prediction)

In [17]:
body_pred

[{'sequence': '\xa0Le couturier français Ted Lapidus est mort lundi 29 décembre à \xa0\xa0\xa0l\'âge de 79 ans à l\'hôpital de Cannes (Alpes-Maritimes), a-t-on appris \xa0\xa0\xa0auprès de sa famille. "Il souffrait depuis plusieurs années d\'une \xa0\xa0\xa0leucémie et est décédé des suites d\'une insuffisance respiratoire", a \xa0\xa0\xa0déclaré à l\'AFP sa soeur et couturière Rose Torrente-Mett.\xa0\xa0Le couturier, très prisé dans les 1960 et 1970, doit être \xa0\xa0\xa0inhumé vendredi 2 janvier au cimetière du père Lachaise à Paris.\xa0\xa0Edmond Lapidus, dit Ted Lapidus, était né le 23 juin 1929 à Paris. Ce \xa0\xa0\xa0fils de tailleurs russe émigré "avait ouvert sa maison en 1958 et Annabelle \xa0\xa0\xa0Buffet avait été un de ses premiers mannequins", a raconté à l\'AFP sa soeur \xa0\xa0\xa0en ajoutant qu\'il avait ainsi habillé Brigitte Bardot ou Alain Delon. "Ted \xa0\xa0\xa0a été le premier couturier de la nouvelle vague (...) le monde entier le \xa0\xa0\xa0connaissait", a-t-